# Prepare data for information retrieval fine-tuning 

## Map Bible verse to body segments with the relevant citations

In [8]:
import re,json,os
import pandas as pd 
from tqdm import tqdm 
import string 
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

def clean_text(s): 
    s = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>,","",s) # \d+\^PAGE[S]*\^MISSING"
    s = re.sub(r"\s+"," ",s)
    return s.strip(" ").lower()
    ret_str = []
    for token in s.lower().split(): 
        ret_str.append(token.strip(string.punctuation))
    return " ".join(ret_str).strip()
with open('../assets/corpora.json',"r") as file: 
    eras = json.load(file)
    

In [ ]:
relevant = {}
c_type = "verse"

for era in eras: 
    with open(f"../assets/citations/{era}_citation_segments.json") as file:
        c_to_seg = json.load(file)
    seg_to_c = {}
    for cited, segments in c_to_seg.items():
        if "Ibidem" in cited or "Verse" in cited: continue 
        cited = " ".join(cited.split("-"))
        if re.search(r"\d+ \d+",cited):
            cited = cited.split(" ")
            if c_type == "verse": 
                cited = " ".join(cited[:-2]) +" " + ".".join(cited[-2:])
            else: cited = " ".join(cited)
        for s in segments: 
            seg_id = (s.split(",")[0],int(s.split(",")[1]))
            if seg_id not in seg_to_c: 
                seg_to_c[seg_id] = []
            seg_to_c[seg_id].append(cited)
    # print(era, len(c_to_seg),'citations',len(seg_to_c),"segments")

    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None)
        for idx, tcpID in enumerate(text[0]):
            sidx = str(text[1][idx])
            
            close_to_citation = False 
            citation_sidx = []
            # window of 2 segments 
            if (tcpID,int(sidx)) in seg_to_c:
                close_to_citation = True
                citation_sidx.append(int(sidx))
            else: 
                i = 1 
                if (tcpID,int(sidx)-i) in seg_to_c:
                    close_to_citation = True
                    citation_sidx.append(int(sidx)-i)
                elif (tcpID,int(sidx)+i) in seg_to_c:
                    close_to_citation = True 
                    citation_sidx.append(int(sidx)+i)
            if close_to_citation: 
                s = clean_text(text[6][idx])
                if len(s.split(" "))< 5: continue # at least 5 words long
                if s not in relevant: 
                    relevant[s] = ({},{}) # citations, location 
                relevant[s][1][(tcpID,sidx)] = None  
                for entry in citation_sidx: 
                    for c in seg_to_c[(tcpID,entry)]:
                        relevant[s][0][c] = None 
    print("Finished processing",era,len(set(relevant)),'unique passages')



In [3]:
output = {}
for s,r in relevant.items(): 
    output[s] = (list(r[0].keys()), list(r[1].keys()))
with open(f"../../segments/all_relevant.json","w+") as file: 
    json.dump(output, file)
del output

## Get non-citation segments 

In [9]:
# Body Texts  
from tqdm import tqdm 

body = {}
margin_p = {}

for era in eras: 
    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "body" not in fp: continue 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None)
        text = text.to_dict(orient="records")
        for item in text:
            s = clean_text(item[6])
            # if s in relevant: continue 
            tcpID, sidx = item[0], item[1]
            if not isinstance(s,str): continue 
            if len(s) == 0: continue 
            if s not in body: 
                body[s] = []# locations 
            body[s].append((tcpID,str(sidx),'In-Text')) 

    print(f"Finished {era}: Total {len(body)} body passages.")

    # Marginalia
    
    for fp in tqdm(os.listdir(f"../../SERMONS_APP/db/data/{era}")):
        if "margin" not in fp: continue 
        margin = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{fp}", header=None)
        margin = margin.to_dict(orient="records")
        for m in margin:
            s = m[3]
            if isinstance(s,float): continue
            s = clean_text(s)
            tcpID, sidx, nidx = m[0], m[1], m[2]
            if s not in margin_p: 
                margin_p[s] = [] # locations 
            margin_p[s].append((tcpID,str(sidx),"Note " + str(nidx)))

    print(f"Finished {era} marginalia: Total {len(margin_p)} margin passages.")

100%|██████████| 13/13 [00:02<00:00,  6.01it/s]


Finished pre-Elizabeth: Total 59415 body passages.


100%|██████████| 13/13 [00:00<00:00, 105.63it/s]


Finished pre-Elizabeth marginalia: Total 5235 margin passages.


100%|██████████| 13/13 [00:16<00:00,  1.24s/it]


...

Finished JamesII marginalia: Total 418776 margin passages.


100%|██████████| 20/20 [00:26<00:00,  1.33s/it]


Finished WilliamAndMary: Total 6125769 body passages.


100%|██████████| 20/20 [00:00<00:00, 42.63it/s]

Finished WilliamAndMary marginalia: Total 435906 margin passages.


In [11]:
with open(f"../../segments/all_unique.json","w+") as file: 
    json.dump(body,file)

with open(f"../../segments/all_unique_marginalia.json","w+") as file: 
    json.dump(margin_p,file)

In [5]:
del body, margin_p